## Analyzing Fairness of the Model
**NOTE:** Feel free to change the code template. 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!tar -xvf "/content/drive/MyDrive/Colab Notebooks/2023-SP-101-hw4-yam64/part1.tar.gz"


Streaming output truncated to the last 5000 lines.
part1/14_0_2_20170103201051263.jpg
part1/30_1_0_20170104170158777.jpg
part1/62_0_0_20170104212928133.jpg
part1/27_1_0_20170103230354912.jpg
part1/65_1_0_20170110123252108.jpg
part1/60_1_0_20170110151441645.jpg
part1/44_1_3_20170109141426511.jpg
part1/62_0_3_20170104220837477.jpg
part1/25_1_0_20170111182452788.jpg
part1/10_1_0_20170109204244904.jpg
part1/27_1_0_20170105183939496.jpg
part1/1_0_2_20161219212557190.jpg
part1/9_0_0_20170110221659430.jpg
part1/67_1_0_20170110140730634.jpg
part1/62_1_0_20170110160643751.jpg
part1/1_1_3_20161220220534186.jpg
part1/39_0_2_20170104204400674.jpg
part1/27_1_3_20170104223400455.jpg
part1/37_0_4_20170104000748917.jpg
part1/2_0_2_20161219162357438.jpg
part1/2_0_2_20161219222714623.jpg
part1/52_0_2_20170104184356222.jpg
part1/60_1_0_20170110122626700.jpg
part1/32_1_0_20170103181503793.jpg
part1/41_0_0_20170109012220690.jpg
part1/1_0_4_20161221201411850.jpg
part1/60_0_0_20170111203255875.jpg
part1/1_1_

In [3]:
path= '/content/drive/MyDrive/Colab Notebooks/2023-SP-101-hw4-yam64/part1'

In [7]:
import cv2
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [17]:
# Function to check if a string can be converted to an integer
def is_integer(string):
    try:
        int(string)
        return True
    except ValueError:
        return False

def load_dataset(path):
    data = []
    labels = []
    for image in os.listdir(path):
        img_path = os.path.join(path, image)
        if img_path.endswith('.jpg'):
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (100, 100)) 
            label = (image.split('/')[-1].split('_')[2])
            # Check if the extracted label is an integer
            if is_integer(label):
                label = int(label)
                data.append(img)
                labels.append(label)
    return np.array(data), np.array(labels)

def split_train_test(X_train, y_train):
  test_size = 0.2
  return train_test_split(X_train, y_train, test_size = test_size, random_state=42)

def load_model():
  return InceptionV3(weights='imagenet', include_top=False, input_shape=(100, 100, 3))

In [18]:
#Note: Replace "/path/to/UTKFace" with the actual path to the UTKFace dataset on your local machine.
X_train, y_train = load_dataset('part1')

X_train = X_train.astype('float32') / 255.0
y_train = tf.keras.utils.to_categorical(y_train, num_classes=5)

X_train, X_val, y_train, y_val = split_train_test(X_train, y_train)
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

(8108, 100, 100, 3)
(8108, 5)
(2027, 100, 100, 3)
(2027, 5)


In [19]:
base_model = load_model()
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(5, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [20]:
# Define hyperparameters
loss_func = 'categorical_crossentropy'
optim = 'SGD'
batch_size = 20
epochs = 15

model.compile(optimizer=optim, loss=loss_func, metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val))

Epoch 1/15
406/406 [==============================] - 964s 2s/step - loss: 1.2086 - accuracy: 0.5522 - val_loss: 1.1098 - val_accuracy: 0.5900
Epoch 2/15
406/406 [==============================] - 950s 2s/step - loss: 1.0222 - accuracy: 0.6170 - val_loss: 1.0054 - val_accuracy: 0.6295
Epoch 3/15
406/406 [==============================] - 1016s 3s/step - loss: 0.8777 - accuracy: 0.6759 - val_loss: 0.9858 - val_accuracy: 0.6295
Epoch 4/15
406/406 [==============================] - 938s 2s/step - loss: 0.7389 - accuracy: 0.7221 - val_loss: 0.9002 - val_accuracy: 0.6828
Epoch 5/15
406/406 [==============================] - 924s 2s/step - loss: 0.5942 - accuracy: 0.7801 - val_loss: 1.1390 - val_accuracy: 0.6226
Epoch 6/15
406/406 [==============================] - 876s 2s/step - loss: 0.4978 - accuracy: 0.8185 - val_loss: 0.9644 - val_accuracy: 0.6961
Epoch 7/15
406/406 [==============================] - 867s 2s/step - loss: 0.4108 - accuracy: 0.8488 - val_loss: 1.0850 - val_accuracy: 0.663

**Validation**

In [21]:
# Load the validation set and preprocess it
X_val, y_val = load_dataset('part1')
X_val = X_val.astype('float32') / 255.0
y_val = tf.keras.utils.to_categorical(y_val, num_classes=5)

# Make predictions using the trained model
y_pred = model.predict(X_val)

# Convert the one-hot encoded vectors to integer labels
y_val_labels = np.argmax(y_val, axis=1)
y_pred_labels = np.argmax(y_pred, axis=1)

# Initialize an empty dictionary to store the results
results_dict = {}

# Compute the number of positive/negative predictions for each race and store in the dictionary
races = ['White', 'Black', 'Asian', 'Indian', 'Others']
for i, race in enumerate(races):
    race_indices = np.where(y_val_labels == i)[0]
    race_predictions = y_pred_labels[race_indices]
    num_positives = np.sum(race_predictions == i)
    num_negatives = len(race_indices) - num_positives
    results_dict[race] = {'positive': num_positives, 'negative': num_negatives}

print(results_dict)

##TODO: Evaluate the 'statistical parity' of the model by evaluating the conditional probability of positive predictions across each race.


##TODO: Evaluate the 'equalized odds' of the model by evaluating the conditional probability of positive predictions across each race.


##TODO: Evaluate the 'calibration' of the model by evaluating the conditional probability of positive predictions across each race.

317/317 [==============================] - 159s 496ms/step
{'White': {'positive': 4911, 'negative': 485}, 'Black': {'positive': 262, 'negative': 160}, 'Asian': {'positive': 1352, 'negative': 351}, 'Indian': {'positive': 953, 'negative': 540}, 'Others': {'positive': 948, 'negative': 173}}


In [22]:
#Evaluate the 'statistical parity' of the model by evaluating the conditional probability of positive predictions across each race.
for race in races:
    race_positives = results_dict[race]['positive']
    race_negatives = results_dict[race]['negative']
    race_total = race_positives + race_negatives
    
    # Calculate the conditional probability of positive predictions for the race
    race_prob_positives = race_positives / race_total
    print(f'{race}: {race_prob_positives}')


White: 0.9101186063750927
Black: 0.6208530805687204
Asian: 0.7938931297709924
Indian: 0.6383121232417951
Others: 0.8456735057983943


In [23]:
# Evaluate the 'equalized odds' of the model by evaluating the conditional probability of positive predictions across each race.
for race in races:
    race_indices = np.where(y_val_labels == races.index(race))[0]
    race_predictions = y_pred_labels[race_indices]
    num_true_positives = np.sum((race_predictions == races.index(race)) & (y_val_labels[race_indices] == races.index(race)))
    num_false_positives = np.sum((race_predictions == races.index(race)) & (y_val_labels[race_indices] != races.index(race)))
    true_positive_prob = num_true_positives / (num_true_positives + num_false_positives)
    print(f"Race: {race}, Probability of true positive prediction: {true_positive_prob}")


Race: White, Probability of true positive prediction: 1.0
Race: Black, Probability of true positive prediction: 1.0
Race: Asian, Probability of true positive prediction: 1.0
Race: Indian, Probability of true positive prediction: 1.0
Race: Others, Probability of true positive prediction: 1.0


In [24]:
#Evaluate the 'calibration' of the model by evaluating the conditional probability of positive predictions across each race
for race in races:
    race_indices = np.where(y_val_labels == i)[0]
    race_predictions = y_pred_labels[race_indices]
    num_positives = np.sum(race_predictions == i)
    positive_prob = num_positives / len(race_indices)
    print(f"Race: {race}, Probability of positive prediction: {positive_prob}")


Race: White, Probability of positive prediction: 0.8456735057983943
Race: Black, Probability of positive prediction: 0.8456735057983943
Race: Asian, Probability of positive prediction: 0.8456735057983943
Race: Indian, Probability of positive prediction: 0.8456735057983943
Race: Others, Probability of positive prediction: 0.8456735057983943
